# Notebook 4 - Education and Salary - a first overview.

This notebook aims to create a single dataframe collecting data from the _Census.Gov_ page with information on:
1. Education Levels - Namely the percentage of Over25 that hold a Bachelor's degree
2. Education Institution Presence - from the Firms API we analysed the "educational services" category (number 61) that showed us per each county how many educ. centres were present (this includes kindergarten, schools of any level and university)
3. Salary - The total income of the county

**PLEASE NOTE** While the aim of our research is to show an uniform patter in all states of the United States, we will only take into consideration the state of California, in order to speed up the output results.


We start by importing the necessary libraries, retrieving the data from the API and analysing only the state of California for the sake of notebook size, we will proceed to a further analysis in a later step of our project

In [43]:
#Required imports for the project
import requests # for api requests
import pandas as pd #tabular data

from bs4 import BeautifulSoup

In [44]:
# !! REMINDER TO TAKE API KEY OUT OF CODE BEFORE SUBMITTING !!
api_key = ""

In [45]:
#creating a dataframe function
def json_to_dataframe(response):
    return pd.DataFrame(response.json()[1:], columns=response.json()[0])

In [46]:
#Setting up the API Query parameters for the educational services industry
params1 = {"NAICS2017" : 61}

In [47]:
#Requesting the json file from the census website using the api key
url = "https://api.census.gov/data/2017/ecnbasic?get=NAICS2017_LABEL,NAICS2017,GEO_ID,FIRM&for=county:*&key={}".format(api_key)
response = requests.request("GET", url, params=params1)

In [48]:
response.text[0:77]

'[["NAICS2017_LABEL","NAICS2017","GEO_ID","FIRM","NAICS2017","state","county"]'

In [49]:
#Converting ecn data frame into pandas data frame
educational_services = json_to_dataframe(response)
df = pd.DataFrame(data = educational_services)
df['FIRM'] = df['FIRM'].astype(int)
df['state'] = df['state'].astype("string")
df_california1 = df[df['state'] == '06']

In [50]:
# Top 5 counties with largest number of educational services
df_c_sort = df_california1\
.sort_values(by=['FIRM'], ascending=False)\
.head(10)
print('Top 10 counties with the most amount of educational services in California')
df_c_sort

Top 10 counties with the most amount of educational services in California


,NAICS2017_LABEL,NAICS2017,GEO_ID,FIRM,NAICS2017,state,county
231,Educational services,61,0500000US06037,2787,61,06,037
283,Educational services,61,0500000US06059,1187,61,06,059
329,Educational services,61,0500000US06073,1010,61,06,073
302,Educational services,61,0500000US06085,822,61,06,085
212,Educational services,61,0500000US06001,693,61,06,001
217,Educational services,61,0500000US06075,470,61,06,075
213,Educational services,61,0500000US06067,334,61,06,067
318,Educational services,61,0500000US06081,323,61,06,081
300,Educational services,61,0500000US06065,313,61,06,065
206,Educational services,61,0500000US06013,295,61,06,013


In [51]:
#Setting the params
params2 = {"state" : "06"}

In [52]:
#Getting the ACS data
#Requesting the json file from the census website using the api key 
url = "https://api.census.gov/data/2017/acs/acs1/profile?get=DP02_0064PE,DP02_0088PE,DP02_0123PE&for=county&key={}".format(api_key)
response2 = requests.request("GET", url)

In [53]:
#Seeing what the columns for the data are
response2.text[0:61]

'[["DP02_0064PE","DP02_0088PE","DP02_0123PE","state","county"]'

## Meaning of variables
- **DP02_0064PE** = Percent!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree
- **DP02_0088PE** = Percent!!PLACE OF BIRTH!!Total population!!Native!!Born in United States 
- **DP02_0123PE** = Percent!!ANCESTRY!!Total population!!American

In [54]:
pop_chars = json_to_dataframe(response2)
df = pd.DataFrame(data = pop_chars)
df_california2 = df[df['state'] == '06']

In [55]:
#Merging the two datasets
merged_census = pd.merge(df_california1, df_california2, on='county')
merged_census.head()
#drop state_y column and rename state_x to state
merged_census = merged_census.drop(columns=['state_y', 'NAICS2017'])
merged_census = merged_census.rename(columns={'state_x': 'state', 'FIRM':'Number of Educational Institutions', 'DP02_0064PE':'Percent of Population with a Bachelor\'s Degree'})
# The last two columns are currently unneeded but will be later used for contextual analysis.
#Drop the last two columns
merged_census = merged_census.drop(columns=['DP02_0088PE', 'DP02_0123PE'])
merged_census.head()


,NAICS2017_LABEL,GEO_ID,Number of Educational Institutions,state,county,Percent of Population with a Bachelor's Degree
0,Educational services,0500000US06047,13,06,047,8.7
1,Educational services,0500000US06033,8,06,033,9.5
2,Educational services,0500000US06115,0,06,115,13.9
3,Educational services,0500000US06013,295,06,013,26.7
4,Educational services,0500000US06099,71,06,099,12.0


## Getting the Code on Salary

In [56]:
#importing the excel file called "GDP_County_2021_NotPerCapita.xlsx" and transforming it into a pandas dataframe
###change the directory to the location of the excel file on your computer

gdp = pd.read_excel("file:/Users/albertomartino/Desktop/LSE/DS105/Summatives/Project/LSE-DS105L-Data-Detectives/notebooks/GDP_County_2021_NotPerCapita.xlsx")
gdp.head(20)

,StateName,CountyName,GDP
0,Alabama,Autauga,1502153
1,Alabama,Baldwin,7830237
2,Alabama,Barbour,709459
3,Alabama,Bibb,392249
4,Alabama,Blount,997835
5,Alabama,Bullock,236820
6,Alabama,Butler,652486
7,Alabama,Calhoun,4245199
8,Alabama,Chambers,844188
9,Alabama,Cherokee,533432


In [57]:
codes = pd.read_excel("file:/Users/albertomartino/Desktop/LSE/DS105/Summatives/Project/LSE-DS105L-Data-Detectives/notebooks/fips2county.xlsx")
codes.head(20)
#In CountyFIPS column, if the value is under 10000, add a 0 in front of it
codes['CountyFIPS'] = codes['CountyFIPS'].apply(lambda x: '{0:0>5}'.format(x))
codes.head(20) 

,StateFIPS,CountyFIPS_3,CountyName,StateName,CountyFIPS,StateAbbr,STATE_COUNTY
0,1,1,Autauga,Alabama,01001,AL,AL | AUTAUGA
1,1,3,Baldwin,Alabama,01003,AL,AL | BALDWIN
2,1,5,Barbour,Alabama,01005,AL,AL | BARBOUR
3,1,7,Bibb,Alabama,01007,AL,AL | BIBB
4,1,9,Blount,Alabama,01009,AL,AL | BLOUNT
5,1,11,Bullock,Alabama,01011,AL,AL | BULLOCK
6,1,13,Butler,Alabama,01013,AL,AL | BUTLER
7,1,15,Calhoun,Alabama,01015,AL,AL | CALHOUN
8,1,17,Chambers,Alabama,01017,AL,AL | CHAMBERS
9,1,19,Cherokee,Alabama,01019,AL,AL | CHEROKEE


In [60]:
#For each row of GDP get the CountyFIPS corresponding to the county name and state in the codes dataframe
#Then add the CountyFIPS to the GDP dataframe

### TO - DO

# 🚧 Work in Progress - Data Visualisation: Is there a relation between GDP and Education Level? 🚧
@Seyi and @Alua will complete this section soon.